<a href="https://colab.research.google.com/github/danalex7212/DataScience/blob/main/SVM_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimal Margin Classifier (Primal Form)

In [136]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [137]:
mnist = tf.keras.datasets.mnist

In [138]:
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

In [139]:
index_train = np.where((train_labels == 2) | (train_labels == 6))
index_test = np.where((test_labels == 2) | (test_labels == 6) )
train_images_26 = train_images[index_train]
train_images_26 = train_images_26.reshape((len(train_images_26),train_images_26[1].size))
train_labels_26 = train_labels[index_train].astype('int')
test_images_26 = test_images[index_test]
test_images_26 = test_images_26.reshape((len(test_images_26),test_images_26[1].size))
test_labels_26 = test_labels[index_test].astype('int')
train_labels_26[ np.where(train_labels_26 == 2 )] = -1
train_labels_26[ np.where(train_labels_26 == 6 )] = 1
test_labels_26[ np.where(test_labels_26 == 2 )] = -1
test_labels_26[ np.where(test_labels_26 == 6 )] = 1
(train_images_26)= train_images_26[range(1000)].astype(np.float64)
(train_labels_26)= train_labels_26[range(1000)].astype(np.float64)

train_images_26 = train_images_26/255
test_images_26 = test_images_26/255



In [140]:
import cvxpy as cp
w = cp.Variable(((train_images_26).shape[1]))

obj = cp.Minimize(cp.sum_squares(w))
b = cp.Variable()
const = cp.multiply(train_labels_26,train_images_26*w+b)
constraints = [const>=1]

prob=cp.Problem(obj,constraints)
prob.solve()
print("status:", prob.status)    
print("optimal value", prob.value) 



status: optimal
optimal value 7.207983941130411


In [141]:
incorr = 0
for j in range(len(train_images_26)): 
      wxtest = np.matmul(w.value,train_images_26[j]) + b.value
      if wxtest*train_labels_26[j]<=0:
        incorr += 1
print("b value",b.value)


primal_training_error=((incorr)/len(train_images_26))
print("training error :",primal_training_error)

b value -0.6002426560740116
training error : 0.0


In [142]:
incorr = 0
for j in range(len(test_images_26)): 
      wxtest = np.matmul(w.value,test_images_26[j]) + b.value
      if wxtest*test_labels_26[j]<=0:
        incorr += 1


primal_testing_error = ((incorr)/len(test_images_26))
print("testing error :",primal_testing_error)

testing error : 0.024623115577889446




# Dual Form of Optimal Marginal Classifier

In [143]:
import cvxpy as cp
a = cp.Variable((train_labels_26.shape[0]))

def dot_mat_vector(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= product(x[i],x[j])
  return mat

def product(c,d):
  return np.dot(c,d)
  

def dot_mat(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= x[i]*x[j]
  return mat

mat_x = np.zeros((len(train_images_26), len(train_images_26))).astype(np.float64)
mat_x = dot_mat_vector(train_images_26)

mat_y = np.zeros((len(train_labels_26), len(train_labels_26)))
mat_y = dot_mat(train_labels_26)

h = np.zeros((train_images_26.shape[0], train_images_26.shape[0])).astype(np.float64)
h = np.multiply(mat_x,mat_y)


obj = cp.Maximize(cp.sum(a)-0.5*cp.quad_form(a,h))
c2 = [a@train_labels_26.T >= 0]
a_const = [a[i] >=0 for i in range(train_labels_26.shape[0])]

constraints = c2+a_const
prob=cp.Problem(obj,constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)




status: optimal
optimal value 3.6039919705652093


In [144]:
w_dual = np.zeros(train_images_26.shape[1])

for i in range(len(train_images_26)):
  w_dual += a.value[i]*train_labels_26[i]*train_images_26[i]

In [145]:
max = -np.inf
min = np.inf
for i in range(len(train_images_26)):
  if ( train_labels_26[i] == -1):
    if np.dot(train_images_26[i],w_dual.T) > max :
      max = np.dot(train_images_26[i],w_dual.T)
  else:
    if np.dot(train_images_26[i],w_dual.T) < min :
      min = np.dot(train_images_26[i],w_dual.T)

b_dual = (-1*(max + min))/2


In [146]:
incorr = 0

for j in range(len(train_images_26)): 
      wxtest = np.matmul(w_dual,train_images_26[j]) + b_dual
      if wxtest*train_labels_26[j]<=0:
        incorr += 1
dual_training_error = ((incorr)/len(train_images_26))
print("training error :",dual_training_error)

training error : 0.0


In [147]:
incorr = 0
for j in range(len(test_images_26)): 
      wxtest = np.matmul(w_dual,test_images_26[j]) + b_dual
      if wxtest*test_labels_26[j]<=0:
        incorr += 1
dual_testing_error = ((incorr)/len(test_images_26))
print("testing error :",dual_testing_error)

testing error : 0.024623115577889446


#Gaussian Kernel SVM 

In [148]:
a_kernel = cp.Variable((train_labels_26.shape[0]))


def dot_mat_vector(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= product(x[i],x[j])
  return mat

def product(c,d):
  
  sigma = 1
  num = -1*np.sum(np.square(c-d))
  denom = 2*sigma**2
  return np.exp((num/denom))
  

def dot_mat(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= x[i]*x[j]
  return mat

mat_x = np.zeros((len(train_images_26), len(train_images_26))).astype(np.float64)
mat_x = dot_mat_vector(train_images_26)

mat_y = np.zeros((len(train_labels_26), len(train_labels_26)))
mat_y = dot_mat(train_labels_26)

h = np.zeros((train_images_26.shape[0], train_images_26.shape[0])).astype(np.float64)
h = np.multiply(mat_x,mat_y)


obj = cp.Maximize(cp.sum(a_kernel)-0.5*cp.quad_form(a_kernel,h))
c2 = [a_kernel@train_labels_26.T == 0]
a_const = [a_kernel[i] >=0 for i in range(train_labels_26.shape[0])]

constraints = c2+a_const
prob=cp.Problem(obj,constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

status: optimal
optimal value 499.5828224579079


In [149]:

def summation(x):
  w_sum = 0
  
  for i in range(len(train_labels_26)):
    w_sum += train_labels_26[i]*a_kernel.value[i]*product(train_images_26[i],x)
  return w_sum
max = -np.inf
min = np.inf
for i in range(len(train_images_26)):
  if ( train_labels_26[i] == -1):
    if summation(train_images_26[i]) > max :
      max = summation(train_images_26[i])
  else:
    if summation(train_images_26[i]) < min :
      min = summation(train_images_26[i])

b_kernel = (-1*(max + min))/2

In [150]:
incorr = 0
for j in range(len(train_images_26)): 
      wxtest = summation(train_images_26[j]) + b_kernel
      if wxtest*train_labels_26[j]<=0:
        incorr += 1
kernel_training_error = ((incorr)/len(train_images_26))
print("training error :",kernel_training_error)

training error : 0.0


In [151]:
incorr = 0

for j in range(len(test_images_26)): 
      wxtest = summation(test_images_26[j]) + b_kernel
      if wxtest*test_labels_26[j]<=0:
        incorr += 1
kernel_testing_error = ((incorr)/len(test_images_26))
print("testing error :",kernel_testing_error)

testing error : 0.5185929648241207


#K- Nearest Neighbor

In [152]:
#KNN for k = 3


k = 3
incorr = 0  #incorrect label is initialized as zero
for i in range(0,len(train_images_26)):
        dist = [] #initialize distance vector as empty array
        dist = np.sqrt(np.sum((train_images_26[i]-train_images_26)**2,axis = 1)) #calculate distance of training sample from all of the rest of the training data
        rank = np.argsort(dist) #sort the distances and return the indices of the sorted array using numpy argsort function
        vote = 0 #initialize majority vote as neutral 0 ,neither +1 nor -1
        for j in range(k):
            vote += train_labels_26[rank[j]] #sum of the training lablels of k least distances will give the predicted label for the sample.( -1 and 1 will cancel each other)
        if vote*train_labels_26[i] < 0: #if sign of the predicted and actual label is different ,there product is less than zero and it will be incorrect prediction
          incorr +=1  #increment counter for incorrect prediction 

k3_training_error = ((incorr)/len(train_images_26))       
print("training error :",k3_training_error,"Incorrect classified : ",incorr) #print training error

incorr = 0  #incorrect label is initialized as zero
for i in range(0,len(test_images_26)):
        dist = [] #initialize distance vector as empty array
        dist = np.sqrt(np.sum((test_images_26[i]-train_images_26)**2,axis = 1)) #calculate distance of testing sample from all of the rest of the training data
        rank = np.argsort(dist) #sort the distances and return the indices of the sorted array using numpy argsort function
        vote = 0 #initialize majority vote as neutral 0 ,neither +1 nor -1
        for j in range(k):
            vote += train_labels_26[rank[j]]#sum of the training lablels of k least distances will give the predicted label for the sample. ( -1 and 1 will cancel each other)
        if vote*test_labels_26[i] < 0: #if sign of the predicted and actual label is different ,there product is less than zero and it will be incorrect prediction
          incorr +=1  #increment counter for incorrect prediction 
        
k3_testing_error = ((incorr)/len(test_images_26))
print("testing error :",k3_testing_error,"Incorrect classified : ",incorr) #print testing error

training error : 0.002 Incorrect classified :  2
testing error : 0.006030150753768844 Incorrect classified :  12


In [153]:
#KNN for k = 5


k = 5
incorr = 0  #incorrect label is initialized as zero
for i in range(0,len(train_images_26)):
        dist = [] #initialize distance vector as empty array
        dist = np.sqrt(np.sum((train_images_26[i]-train_images_26)**2,axis = 1)) #calculate distance of training sample from all of the rest of the training data
        rank = np.argsort(dist) #sort the distances and return the indices of the sorted array using numpy argsort function
        vote = 0 #initialize majority vote as neutral 0 ,neither +1 nor -1
        for j in range(k):
            vote += train_labels_26[rank[j]] #sum of the training lablels of k least distances will give the predicted label for the sample.( -1 and 1 will cancel each other)
        if vote*train_labels_26[i] < 0: #if sign of the predicted and actual label is different ,there product is less than zero and it will be incorrect prediction
          incorr +=1  #increment counter for incorrect prediction 

k5_training_error = ((incorr)/len(train_images_26))        
print("training error :",((incorr)/len(train_images_26)),"Incorrect classified : ",incorr) #print training error

incorr = 0  #incorrect label is initialized as zero
for i in range(0,len(test_images_26)):
        dist = [] #initialize distance vector as empty array
        dist = np.sqrt(np.sum((test_images_26[i]-train_images_26)**2,axis = 1)) #calculate distance of testing sample from all of the rest of the training data
        rank = np.argsort(dist) #sort the distances and return the indices of the sorted array using numpy argsort function
        vote = 0 #initialize majority vote as neutral 0 ,neither +1 nor -1
        for j in range(k):
            vote += train_labels_26[rank[j]]#sum of the training lablels of k least distances will give the predicted label for the sample. ( -1 and 1 will cancel each other)
        if vote*test_labels_26[i] < 0: #if sign of the predicted and actual label is different ,there product is less than zero and it will be incorrect prediction
          incorr +=1  #increment counter for incorrect prediction 
        
k5_testing_error = ((incorr)/len(test_images_26))
print("testing error :",((incorr)/len(test_images_26)),"Incorrect classified : ",incorr) #print testing error

training error : 0.006 Incorrect classified :  6
testing error : 0.007035175879396985 Incorrect classified :  14


#Summary of results

Linear Classification by optimal marginal classifer and dual form SVM produce good testing error . Gaussian Kernel produces bad testing error with sigma = 1 . However increasing sigma gives better testing error result for Kernel SVM with gaussian kernel. KNN for K=3 and K=5 produces the best result on the given dataset.

In [154]:
from tabulate import tabulate
table = [["Test","Training Error","Testing Error"],\
         ["Primal form",primal_training_error,primal_testing_error],\
         ["Dual form",dual_training_error,dual_testing_error],\
         ["Kernel SVM ",kernel_training_error,kernel_testing_error],\
         ["KNN K=3 ",k3_training_error,k3_testing_error],\
         ["KNN K=5 ",k5_training_error,k5_testing_error]]
#print(tabulate(table))
print(tabulate(table,headers="firstrow",tablefmt="fancy_grid"))

╒═════════════╤══════════════════╤═════════════════╕
│ Test        │   Training Error │   Testing Error │
╞═════════════╪══════════════════╪═════════════════╡
│ Primal form │            0     │      0.0246231  │
├─────────────┼──────────────────┼─────────────────┤
│ Dual form   │            0     │      0.0246231  │
├─────────────┼──────────────────┼─────────────────┤
│ Kernel SVM  │            0     │      0.518593   │
├─────────────┼──────────────────┼─────────────────┤
│ KNN K=3     │            0.002 │      0.00603015 │
├─────────────┼──────────────────┼─────────────────┤
│ KNN K=5     │            0.006 │      0.00703518 │
╘═════════════╧══════════════════╧═════════════════╛


#Soft Optimal Margin Classifier using Hinge-loss function (Support Vector Classifer)

In [155]:

import cvxpy as cp
w = cp.Variable(((train_images_26).shape[1]))
b = cp.Variable()
c = 10

e = cp.pos(1-cp.multiply(train_labels_26,(train_images_26@w+b)))
obj = cp.Minimize(cp.sum_squares(w)+c*(1/train_images_26.shape[0])*cp.sum(e))
const = cp.multiply(train_labels_26,train_images_26*w+b)

prob=cp.Problem(obj)
prob.solve()
print("status:", prob.status)     
print("optimal value", prob.value) 
print("optimal var", b.value) 


status: optimal
optimal value 1.5863380044445254
optimal var 0.054512285286243196


In [156]:
incorr = 0
for j in range(len(train_images_26)): 
      wxtest = np.matmul(w.value,train_images_26[j]) + b.value
      if wxtest*train_labels_26[j]<=0:
        incorr += 1
print("b value",b.value)
print("training error :",((incorr)/len(train_images_26)))


b value 0.054512285286243196
training error : 0.017


In [157]:
incorr = 0
for j in range(len(test_images_26)): 
      wxtest = np.matmul(w.value,test_images_26[j]) + b.value
      if wxtest*test_labels_26[j]<=0:
        incorr += 1
print("testing error :",((incorr)/len(test_images_26)))

testing error : 0.03015075376884422


#Soft Margin Kernel SVM

In [158]:
a_kernel = cp.Variable((train_labels_26.shape[0]))

C=5

def dot_mat_vector(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= product(x[i],x[j])
  return mat

def product(c,d):
  
  sigma = 1
  num = -1*(np.sum(np.square(c-d)))
  denom = 2*sigma**2
  return np.exp((num/denom))
  

def dot_mat(x):
  mat = np.zeros((len(x), len(x)))
  for i in range(len(x)):
    for j in range(len(x)):
      mat[i][j]= x[i]*x[j]
  return mat

mat_x = np.zeros((len(train_images_26), len(train_images_26))).astype(np.float64)
mat_x = dot_mat_vector(train_images_26)

mat_y = np.zeros((len(train_labels_26), len(train_labels_26)))
mat_y = dot_mat(train_labels_26)

h = np.zeros((train_images_26.shape[0], train_images_26.shape[0])).astype(np.float64)
h = np.multiply(mat_x,mat_y)


obj = cp.Maximize(cp.sum(a_kernel)-0.5*cp.quad_form(a_kernel,h))
c2 = [a_kernel@train_labels_26.T == 0]
a_const = [a_kernel[i] >=0 for i in range(train_labels_26.shape[0])]
c3 = [a_kernel[i] <=C for i in range(train_labels_26.shape[0])]
constraints = c2+a_const+c3
prob=cp.Problem(obj,constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

status: optimal
optimal value 499.5828224579079


In [159]:

def summation(x):
  w_sum = 0
  
  for i in range(len(train_labels_26)):
    w_sum += train_labels_26[i]*a_kernel.value[i]*product(train_images_26[i],x)
  return w_sum

max = -np.inf
min = np.inf
for i in range(len(train_images_26)):
  if ( train_labels_26[i] == -1):
    if summation(train_images_26[i]) > max :
      max = summation(train_images_26[i])
  else:
    if summation(train_images_26[i]) < min :
      min = summation(train_images_26[i])

b_kernel = (-1*(max + min))/2



In [160]:
incorr = 0
for j in range(len(train_images_26)): 
      wxtest = summation(train_images_26[j]) + b_kernel
      if wxtest*train_labels_26[j]<=0:
        incorr += 1
print("training error :",((incorr)/len(train_images_26)))

training error : 0.0


In [161]:
incorr = 0
for j in range(len(test_images_26)): 
      wxtest = summation(test_images_26[j]) + b_kernel
      if wxtest*test_labels_26[j]<=0:
        incorr += 1
print("testing error :",((incorr)/len(test_images_26)))

testing error : 0.5185929648241207


In [162]:
# import numpy as np
# import seaborn as sns
# import pandas as pd


# cm = sns.light_palette("blue", as_cmap=True)
# x=pd.DataFrame(h)
# x=x.style.background_gradient(cmap=cm)
# display(x)

# def is_pos_def(x):
#     return np.all(np.linalg.eigvals(x) > 0)


# def check_symmetric(a, rtol=1e-05, atol=1e-08):
#     return np.allclose(a, a.T, rtol=rtol, atol=atol)

# print(check_symmetric(h),is_pos_def(h))
# h_real==h